In [ ]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,random_split
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.decomposition import PCA

# Prepare the data

In [ ]:
unprocessed_data =  torch.from_numpy(np.load('Unscaled_data_sorted.npy'))

In [ ]:
train_size=int(0.8*(unprocessed_data.shape[0]))
val_size=len(unprocessed_data) - train_size
print(train_size,val_size)
train_ds,val_ds = random_split(unprocessed_data,[train_size,val_size])

3217614 804404


In [ ]:
train_ds = train_ds[:]
val_ds = val_ds[:]

# Mean-free

In [ ]:
train_ds = train_ds -torch.mean(train_ds,0)
val_ds = val_ds - torch.mean(val_ds,0)
# train_data = train_data- torch.mean(train_data,0)
# shifted_train_data =shifted_train_data- torch.mean(shifted_train_data,0)

# Whitten the data

In [ ]:
pca_whiten = PCA(whiten=True)

train_ds = pca_whiten.fit_transform(train_ds)
val_ds = pca_whiten.fit_transform(val_ds)
# train_data = pca_whiten.fit_transform(train_data)
# shifted_train_data= pca_whiten.fit_transform(shifted_train_data)

In [ ]:
# train_data = torch.tensor(train_data)
train_ds = torch.tensor(train_ds)
val_ds = torch.tensor(val_ds)
# shifted_train_data = torch.tensor(shifted_train_data)

In [ ]:
lag = 8
train_data = train_ds[:-lag]
shifted_train_data = train_ds[lag:]
val_data = val_ds[:-lag]
shifted_val_data = val_ds[lag:]

In [ ]:
torch.std(train_data,0)

tensor([1.0000, 1.0000], dtype=torch.float64)

In [ ]:
shifted_train_data.shape,train_data.shape

(torch.Size([3217606, 2]), torch.Size([3217606, 2]))

In [ ]:
transformed_train_data = torch.hstack((train_data,shifted_train_data))
transformed_val_data = torch.hstack((val_data,shifted_val_data))

# Build the Model

In [ ]:
batch_size=128
train_dl = DataLoader(transformed_train_data,batch_size,shuffle=True)
val_dl = DataLoader(transformed_val_data,batch_size,shuffle=True)

In [ ]:
# for data in train_dl:
#     train_dat = data[:,0:2]
#     shifted_dat = data[:,2:4]
#     make_4_dim(train_data)
# #     print(train_dat.shape,shifted_dat.shape)
# #     print(train_dat,shifted_dat)
#     break

In [ ]:
def make_4_dim(data):
    data=torch.unsqueeze(data,1)
    data = torch.unsqueeze(data,3)
    return data

def make_2_dim(data):
    data=torch.squeeze(data,1)
    data = torch.squeeze(data,2)
    return data

In [ ]:
# out = torch.squeeze(out,2)
# out.shape

In [ ]:
def conv_block1(in_channels,out_channels,kernel_size,stride,padding,pool=False):
    layers = [nn.ConvTranspose2d(in_channels=in_channels,out_channels=out_channels,kernel_size=kernel_size,
                        stride=stride,padding=padding),
              nn.BatchNorm2d(out_channels),
             nn.LeakyReLU(negative_slope=0.2,inplace=True)]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)

def conv_block2(in_channels,out_channels,kernel_size,stride,padding,pool=False):
    layers = [nn.Conv2d(in_channels=in_channels,out_channels=out_channels,kernel_size=kernel_size,
                        stride=stride,padding=padding),
              nn.BatchNorm2d(out_channels),
             nn.LeakyReLU(negative_slope=0.2,inplace=True)]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)
    


In [ ]:
class Resnet9(nn.Module):
      def __init__(self, in_channel, out_channel):
        super().__init__()
        
        # Encode1
        
        self.conv1 = conv_block1(in_channel, 64,kernel_size=4,stride=2,padding=0)         ## 64*6*5
        self.conv2 = conv_block1(64, 128,kernel_size=4,stride=2,padding=0)                ## 128*14*10
        self.conv3 = conv_block1(128, 256,kernel_size=4,stride=2,padding=0)               ##256*30*22
        self.res1 = nn.Sequential(conv_block1(256, 256,kernel_size=3,stride=1,padding=1), ## 256*30*22  
                                  conv_block1(256, 256,kernel_size=3,stride=1,padding=1))  ##256*30*22
        
        #Encode2
        self.conv4 = conv_block2(256, 128,kernel_size=4,stride=2,padding=0)         ## 128*14*10
        self.conv5 = conv_block2(128, 64,kernel_size=4,stride=2,padding=0)                ## 64*6*5
        #self.conv6 = conv_block2(128, 256,kernel_size=4,stride=2,padding=0)               ##256*30*22
        self.res2 = nn.Sequential(conv_block2(64,64,kernel_size=3,stride=1,padding=1), ## 256*30*22  
                                  conv_block2(64,64,kernel_size=3,stride=1,padding=1))  ##256*30*22
        self.conv6 = conv_block2(64,1,kernel_size=4,stride=2,padding=0)                 #1*2*1
        
        #Decode 1
        self.conv7 = conv_block1(in_channel, 64,kernel_size=4,stride=2,padding=0)         ## 64*6*5
        self.conv8 = conv_block1(64, 128,kernel_size=4,stride=2,padding=0)                ## 128*14*10
        self.conv9 = conv_block1(128, 256,kernel_size=4,stride=2,padding=0)               ##256*30*22
        self.res3 = nn.Sequential(conv_block1(256, 256,kernel_size=3,stride=1,padding=1), ## 256*30*22  
                                  conv_block1(256, 256,kernel_size=3,stride=1,padding=1))  ##256*30*22
        
        #Decode 2
        self.conv10 = conv_block2(256, 128,kernel_size=4,stride=2,padding=0)         ## 128*14*10
        self.conv11 = conv_block2(128, 64,kernel_size=4,stride=2,padding=0)                ## 64*6*5
        #self.conv6 = conv_block2(128, 256,kernel_size=4,stride=2,padding=0)               ##256*30*22
        self.res4 = nn.Sequential(conv_block2(64,64,kernel_size=3,stride=1,padding=1), ## 256*30*22  
                                  conv_block2(64,64,kernel_size=3,stride=1,padding=1))  ##256*30*22
        self.conv12 = nn.Conv2d(64,out_channel,kernel_size=4,stride=2,padding=0)                 #1*2*1
        
      def encode(self,in_data):
          out = self.conv1(in_data.float())
          out = self.conv2(out)
          out = self.conv3(out)
          out = self.res1(out)+out
          out = self.conv4(out)
          out = self.conv5(out)
          out = self.res2(out)+out
          out = self.conv6(out)
          return out
           
      def decode(self,lat_data):
          out = self.conv7(lat_data.float())
          out = self.conv8(out)
          out = self.conv9(out)
          out = self.res3(out)+out
          out = self.conv10(out)
          out = self.conv11(out)
          out = self.res4(out)+out
          out = self.conv12(out)
          return out
      
        

In [ ]:
encoder = Resnet9(1,1)

In [ ]:
#encoder

In [ ]:
def training_step(data):
    training_dat = data[:,0:2]
    training_dat = make_4_dim(training_dat)
    shifted_dat = data[:,2:4]
    out = encoder.encode(training_dat)
    out = encoder.decode(out)
    out = make_2_dim(out)
    criterion = nn.MSELoss()
    loss = criterion(out.float(),shifted_dat.float())
    return loss

def evaluate(model,val_loader):
    for data in val_loader:
        val_dat = data[:,0:2]
        val_dat = make_4_dim(val_dat)
        shifted_dat = data[:,2:4]
        out = model.encode(val_dat)
        out = model.decode(out)
        out = make_2_dim(out)
        criterion = nn.MSELoss()
        loss = criterion(out.float(),shifted_dat.float())
        return {'val_loss':loss}

In [ ]:
# #@torch.no_grad()
# criterion = nn.MSELoss()
# optimizer = torch.optim.Adam(encoder.parameters(),lr=0.001)

In [ ]:
# def traing_step(model,batch):
#     out = model.decode(model.encode(batch))
#     loss = nn.MSELoss(out,target)
#     return loss

In [ ]:
epochs =1
model = encoder
max_lr = 0.01
grad_clip =0.1
weight_decay = 1e-4
opt_func = torch.optim.Adam
train_loader =train_dl
val_loader = val_dl

In [ ]:
history = []
optimizer = opt_func(model.parameters(),max_lr,weight_decay=weight_decay)
sched = torch.optim.lr_scheduler.OneCycleLR(optimizer,max_lr,
                                               epochs=epochs,
                                               steps_per_epoch=len(train_loader))
model.train()
train_losses = []

for batch in train_loader:
    loss = training_step(batch)
    train_losses.append(loss)
    loss.backward()
            
            # Gradient clipping
    if grad_clip:
       nn.utils.clip_grad_value_(model.parameters(),
                                         grad_clip)
    optimizer.step()
    optimizer.zero_grad()
            
    sched.step()
        # Validation Phase    
result = evaluate(model,val_loader)
result['train_loss'] = torch.stack(train_losses).mean().item()
print('train_loss{:.4f}'.format(result['train_loss']))
history.append(result)
return history

KeyboardInterrupt: 

In [ ]:
#@torch.no_grad()
def fit_one_cycle(epochs, max_lr, model,train_loader,val_loader,
                 weight_decay,grad_clip,opt_func):
    history = []
    
    # set up custom optimizer with weight decay
    optimizer = opt_func(model.parameters(),max_lr,weight_decay=weight_decay)
    
    # set up one_cycle learning rate scheduler
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer,max_lr,
                                               epochs=epochs,
                                               steps_per_epoch=len(train_loader))
    
    
    # crate the loop for training
    
    for epoch in range(epochs):
        # Training Phase
        model.train()
        train_losses = []
        
        for batch in train_loader:
            loss = training_step(batch)
            train_losses.append(loss)
            loss.backward()
            
            # Gradient clipping
            if grad_clip:
                nn.utils.clip_grad_value_(model.parameters(),
                                         grad_clip)
            optimizer.step()
            optimizer.zero_grad()
            
            sched.step()
        # Validation Phase    
        result = evaluate(model,val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        print('train_loss{:.4f}'.format(result['train_loss']))
        history.append(result)
    return history

In [ ]:
epochs = 40
max_lr = 0.01
grad_clip =0.1
weight_decay = 1e-4
opt_func = torch.optim.Adam

In [ ]:
# for data in val_dl:
#         val_dat = data[:,0:2]
#         val_dat = make_4_dim(val_dat)
#         shifted_dat = data[:,2:4]
#         out = encoder.encode(val_dat)
#         out = encoder.decode(out)
#         #print(out.shape)
#         out = make_2_dim(out)
#         criterion = nn.MSELoss()
#         loss = criterion(out.float(),shifted_dat.float())
#         break

In [ ]:
#loss.type(torch.float32)

In [ ]:
#loss.backward()

In [ ]:
history = [evaluate(encoder,val_dl)]

NameError: name 'evaluate' is not defined

In [ ]:
%%time
history += fit_one_cycle(epochs,max_lr,encoder,train_dl,val_dl,
                        grad_clip=grad_clip,
                        weight_decay = weight_decay,
                        opt_func=opt_func)

In [ ]:
#torch.save(encoder.state_dict(), 'tae-resnet9.pth')
#jovian.commit(project='my_tae', environment=None, outputs=['tae-resnet9.pth'])

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
[jovian] Creating a new project "coolasit16/my_tae"
[jovian] Uploading notebook..
[jovian] Uploading additional outputs...
[jovian] Committed successfully! https://jovian.ai/coolasit16/my-tae


'https://jovian.ai/coolasit16/my-tae'

In [ ]:
#history

history
